In [24]:
class rayleigh_vapor:
    """Generates a vapor dataset from interpolated Rayleigh data"""
    def __init__(self,name=None,varnames=None,varfiles=None, rayleigh_root=None, 
                vapor_root=None, nxyz=None, grid_file=None, force=False, timeout=300,
                remove_spherical_means=[], rmins=[], rmaxes=[], vapor_version=3,
                vector_names=[],vector_files=[], tempdir='.'):

        self.numts=len(varfiles)
        self.varnames=varnames
        self.data_dir = name+'_data'
        self.nvars=len(varnames)
        self.varfiles=varfiles
        self.timeout=timeout
        self.tempdir=tempdir
        
        self.nvec=len(vector_names)
        if (self.nvec > 0):
            self.vector_names = vector_names
            self.vector_files = vector_files
        
        if (vapor_version == 3):
            self.ccmd='vdccreate '
            self.pcmd='raw2vdc '
            self.vaporfile=name+'.vdc'
        else:
            self.ccmd='vdfcreate '
            self.pcmd='raw2vdf -quiet '
            self.vaporfile=name+'.vdf'
        
        if (len(remove_spherical_means) != self.nvars):
            self.remove_spherical_mean=self.nvars*False
        else:
            self.remove_spherical_mean=remove_spherical_means
        
        if (len(rmins) != self.nvars):
            self.zero_rmin=False
            self.rmins=[None]*self.nvars
        else:
            self.zero_rmin=True
            self.rmins=rmins
            
        if (len(rmaxes) != self.nvars):
            self.zero_rmax=False
            self.rmaxes=[None]*self.nvars
        else:
            self.zero_rmax=True
            self.rmaxes=rmaxes       
        
        varstring=' '
        for i in range(self.nvars):
            varstring=varstring+varnames[i]+':'
        if (self.nvec > 0):
            for vn in self.vector_names:
                for n in vn:
                    varstring=varstring+n+':'
        varstring=varstring[0:len(varstring)-1]  # remove trailing ':'
        print(varstring)
        self.varstring=varstring
        self.vapor_root=vapor_root
        self.rayleigh_root=rayleigh_root
        self.nxyz=nxyz
        self.grid_file=grid_file
        if (force == True):
            print('Parameter "force" is set to true.')
            print('Removing: '+self.vaporfile+' > /dev/null')
            print('Removing: '+self.data_dir+' > /dev/null')
            self.destroy_vdc()            
    def create_dataset(self, force=False):
        import subprocess as sp
        res=str(self.nxyz)
        cube_string = res+'x'+res+'x'+res
        cmd1 = 'export PATH=$PATH:'+self.vapor_root+'/bin'
        cmd2 = ' && export LD_LIBRARY_PATH='+self.vapor_root+'/lib:$LD_LIBRARY_PATH && '
        cmd3 = self.ccmd+' -dimension '+cube_string+' -numts '+str(self.numts)
        cmd3 = cmd3+' -vars3d '+self.varstring+' '+self.vaporfile
        creation_cmd=cmd1+cmd2+cmd3
        s=sp.Popen(creation_cmd,shell=True)
        s.wait(timeout=self.timeout)

    def populate_dataset(self):
        import subprocess as sp
        for i in range(self.numts):
            print('Converting data for timestep '+str(i)+' of '+str(self.numts))
            for j in range(self.nvars):
                infile=self.varfiles[i][j]
                ofile=infile+'.cube'
                ofile=self.tempdir+'/temp.cube'
                self.rayleigh_to_cube(infile,ofile,remove_spherical_mean=self.remove_spherical_mean[j], 
                                      rmin=self.rmins[j], rmax=self.rmaxes[j])
                self.cube_to_vdc(ofile,i,j)
            if (self.nvec > 0):
                xfile = self.tempdir+'/x.cube'
                yfile = self.tempdir+'/y.cube'
                zfile = self.tempdir+'/z.cube'
                mfile = self.tempdir+'/m.cube'
                for j in range(self.nvec):
                    vnames = self.vector_names[j]
                    mag=(len(vnames)==4)
                    self.rayleigh_vector_to_cube(self.vector_files[j][i],mag=mag)
                    self.cube_to_vdc(xfile,i, vnames[0])
                    self.cube_to_vdc(yfile,i, vnames[1])
                    self.cube_to_vdc(zfile,i, vnames[2])
                    if (mag):
                        self.cube_to_vdc(mfile,i,vnames[3])
        #Cleanup
        print('Cleaning up temporary files')
        cmd = 'rm -rf '+ofile+' > /dev/null'
        s=sp.Popen(cmd,shell=True)
        s.wait(timeout=self.timeout)
        if (self.nvec > 0):
            for f in [xfile,yfile,zfile,mfile]:
                cmd = 'rm -rf '+f+' > /dev/null'
                s=sp.Popen(cmd,shell=True)
                s.wait(timeout=self.timeout)
        print('Complete.')
                
    def rayleigh_to_cube(self,infile,ofile,remove_spherical_mean=False, rmin=None, rmax=None):
        import subprocess as sp
        cmd1 = 'export PATH=$PATH:'+self.rayleigh_root
        cmd2 = ' &&  interp3d -i '+infile+' -o '+ofile+' -g '+self.grid_file+' -N '+str(self.nxyz)

        if(remove_spherical_mean):
            cmd2=cmd2+" -rsm"

        if(rmin != None):
            cmd2=cmd2+" -rmin "+str(rmin)

        if(rmax != None):
            cmd2=cmd2+" -rmax "+str(rmax)
            
        cmd = cmd1+cmd2
        s=sp.Popen(cmd,shell=True)
        s.wait(timeout=self.timeout)

    def rayleigh_vector_to_cube(self,vfiles, mag=False):
        import subprocess as sp
        rf=vfiles[0]  # r-file
        tf=vfiles[1]  # theta-file
        pf=vfiles[2]  # phi-file
        xfile = self.tempdir+'/x.cube'
        yfile = self.tempdir+'/y.cube'
        zfile = self.tempdir+'/z.cube'
        mfile = self.tempdir+'/m.cube'
        cmd1 = 'export PATH=$PATH:'+self.rayleigh_root
        cmd2 = ' &&  interp3d -ir '+rf+' -it '+tf+' -ip '+pf
        cmd2 = cmd2+' -ox '+xfile+' -oy '+yfile+' -oz '+zfile+' -g '+self.grid_file+' -N '+str(self.nxyz)

        if(mag):
            cmd2=cmd2+" -om "+mfile
            
        cmd = cmd1+cmd2
        #print(cmd)
        s=sp.Popen(cmd,shell=True)
        s.wait(timeout=self.timeout)

    def cube_to_vdc(self,ofile,timeind,varind):
        import subprocess as sp
        if (type(varind) == type(1)):
            varname=self.varnames[varind]
        else:
            varname=varind  # string was passed
        cmd1 = 'export PATH=$PATH:'+self.vapor_root+'/bin'
        cmd2 = ' && export LD_LIBRARY_PATH='+self.vapor_root+'/lib:$LD_LIBRARY_PATH &&'
        cmd3 = self.pcmd+' -ts '+str(timeind)+' -varname '+varname
        cmd3 = cmd3+' '+self.vaporfile+' '+ofile
        cmd = cmd1+cmd2+cmd3
        s=sp.Popen(cmd, shell=True)
        s.wait(timeout=self.timeout)

    def destroy_vdc(self):
        import subprocess as sp
        cmd1 = 'rm -rf '+self.vaporfile +' > /dev/null'
        cmd2 = 'rm -rf '+self.data_dir+' > /dev/null'

        try:
            s=sp.Popen(cmd1,shell=True)
            s.wait(timeout=self.timeout)
            print(cmd1)
        except:
            print('cmd1 error', cmd1)
            s.communicate()
            pass

        print(cmd2)
        try:
            s=sp.Popen(cmd2,shell=True)
            s.wait(timeout=self.timeout)
            print(cmd2)
        except:
            print('cmd2 error', cmd2)
            s.communicate()


def gen_3d_filelist( qcodes, diter, istart,iend, directory='Spherical_3D', ndig=8):
    files = []
    for i in range(istart,iend+diter,diter):
        fstring="{:0>"+str(ndig)+"d}"
        istring = directory+"/"+fstring.format(i)
        f = []
        for q in qcodes:
            qfnt="{:0>"+str(4)+"d}"
            qstr= qfnt.format(q)
            f.append(istring+'_'+qstr)
        files.append(f)
    return files

In [25]:
rundir="/home/feathern/vis/demo2"         # Not necessary, but useful
data_dir=rundir+'/Spherical_3D'          # Where the Rayleigh 3-D output is located
                                         # This is required -- default is './Spherical_3D'
diter = 20  # time-step difference between outputs
first_iter = 268900 # First time-step number to process
last_iter  = 268900 # Last time-step number to process

In [26]:
#First, generate a list of files to use for scalar variables
var_codes = [501, 3]          # quantity codes for temperature and vphi (from Rayleigh)
var_names = ['temp','vphi']    # associated names (user choice)
files=gen_3d_filelist(var_codes,diter,first_iter,last_iter,directory=data_dir)
print(files)

#Note that what is returned here is a 2-D nested list.
#You are not required to use gen_3d_filelist BUT
#You absolutely must respect the 2-D, nested format, even for 1 timestep.  So:
#files[0] = ['quantity 1 @ timestep 0', 'quantity 2 @ timestep 0', ..., 'quantity N @ timestep 0']
#files[1] = ['quantity 1 @ timestep 1', 'quantity 2 @ timestep 1', ..., 'quantity N @ timestep 1']
#...
#files[N] = ['quantity 1 @ timestep N', 'quantity 2 @ timestep N', ..., 'quantity N @ timestep N']

[['/home/feathern/vis/demo2/Spherical_3D/00268900_0501', '/home/feathern/vis/demo2/Spherical_3D/00268900_0003']]


In [27]:
# Many outputs that we might like to visualize have a significant ell=0 or m=0
# component that makes visualization difficult and/or uninteresting.
# FOR SCALAR variables only, these mean values can be substracted, if desired.

# If you would like to remove the spherical  mean, set the remove_means keyword to
# an array of True (remove mean) or False (do not remove) values, 
# 1 element per quantity code.  All elements must
# be specified.

# This is an optional flag, so if you do not want to remove any spherical means,
# omit this flag alltogether.

remove_spherical_means=[True , False]  # Adjust temperature, but leave vphi unchanged

# Similarly, we can remove the longitudinal mean via remove_phi_means.
# This flag works just as remove_sph_means.  
# We don't do this here, but uncomment and modify the line below if you would like to
# remove_phi_means=[False,True]  # This would leave T alone, but remove differential rotation

# Additionally, we can zero out points above and/or below some threshold value in radius.
# This is potentially useful if you are trying to minimize the effect of boundary-layers
# in your models.  This is accomplished through the rmaxes and rmins keywords.  They
# can be used in isolation, or in tandem with one another. Both are optional, so if you 
# don't use them, don't pass them to the dataset creation routine.

# If you would like to zero out r > rmax for each data set, provide an
# array of values below (1 array value for each quantity code).

#IF you want to zero out r > rmax for some quantities, but not for others, 
# you must still specify all array values, setting unused values to None.
# We do not perform this modification here, but
# If we wanted to zero out temperature for r > 1.2 and leave vphi unchanged, we would set
# rmaxes=[1.2, None]  
# For now, however, since we explicitly pass this flag to the creation routine below, we set it,
# explicitly to the value below, which indicates, "do nothing." 
rmaxes=[None, None]

# The rmins flag works the same way.  In the example commented below, we perform 
# no action on temperature, but for vphi, we zero out r < 1.0
#rmins=[None, 1.0].  Again, we don't do this for now, so:
rmins=[None, None]

In [28]:
#Same, but now a list of quantity codes for
# the r,theta,phi components of any vector we want to convert to a Cartesian vector
vqcodes = [1,2,3]        # velocity (r,theta,phi)
wqcodes = [301,302,303]  # vorticity (r,theta,phi)

vfiles=gen_3d_filelist(vqcodes,diter,first_iter,last_iter,directory=data_dir)
wfiles=gen_3d_filelist(wqcodes,diter,first_iter,last_iter,directory=data_dir)
vnames = ['vx', 'vy', 'vz']         # velocity
wnames = ['wx', 'wy', 'wz', 'wmag'] # vorticity, including magnitude 
                                    # the magnitude (optional) is calculated automatically
                                    # useful for coloring vortex lines

vector_names = [vnames, wnames]     # 2-D nested list
vector_files = [vfiles, wfiles]     # 3-D nested list (each 2-D sublist is structured as for scalars above)
print(vector_files)

[[['/home/feathern/vis/demo2/Spherical_3D/00268900_0001', '/home/feathern/vis/demo2/Spherical_3D/00268900_0002', '/home/feathern/vis/demo2/Spherical_3D/00268900_0003']], [['/home/feathern/vis/demo2/Spherical_3D/00268900_0301', '/home/feathern/vis/demo2/Spherical_3D/00268900_0302', '/home/feathern/vis/demo2/Spherical_3D/00268900_0303']]]


In [29]:
#For now, one grid file name must be supplied
#(all files must share the same grid, so only one file is supplied)
gf = rundir+'/Spherical_3D/00268900_grid'

#We must also supply a name for the Vapor dataset.
#Omit the .vdc or .vdf; that is determined based on the Vapor version we choose below.
#Path can be full or local to where you run the notebook
vaporfile = rundir+'/ra_vis_demo' #omit the .vdc or .vdf  

#We will be interpolating the simulation data onto a uniform, Cartesian
#grid of dimension NxNxN
cube_size_N=256

# Next, we need to specify where the Rayleigh repository exists.
# It is assumed that interp3d is compiled and resides at rroot/bin/interp3d
rroot='/home/feathern/devel/forks/Nick/Rayleigh/post_processing/interpolation' 

# While not necessary, it's useful to identify a directory where intermediate files
# will be created, and later deleted.  By default, this is the same directory from
# which the script executes.  Be sure and create the directory beforehand.  If you're
# good with Nick Featherstone writing and deleting whatever he want's from your harddrive,
# there is absolutely no need to pass this variable to the creation routine.
tempdir='/home/feathern/y_r_u_deleting_me'

In [30]:
# Next, we need to specify where Vapor is installed
# Specifically, we want the Vapor "root" directory.
# Looking inside the root directory, you will subdirectories with names like
# bin, lib, include, share ...
vapor_version=2
if (vapor_version == 3):
    vroot='/custom/software/VAPOR3-3.2.0-Linux'   # base directory for Vapor
else:
    vroot='/custom/software/vapor-2.6.0.RC0'    

In [31]:
test = rayleigh_vapor(name=vaporfile,varnames=var_names,varfiles=files,vapor_root=vroot,
                       nxyz=cube_size_N, grid_file=gf, rayleigh_root=rroot, force=True,
                       remove_spherical_means=remove_spherical_means, rmaxes=rmaxes, rmins=rmins, 
                       vapor_version=vapor_version,
                       vector_names=vector_names, vector_files=vector_files, tempdir=tempdir)
test.create_dataset(force=True)

 temp:vphi:vx:vy:vz:wx:wy:wz:wmag
Parameter "force" is set to true.
Removing: /home/feathern/vis/demo2/ra_vis_demo.vdf > /dev/null
Removing: /home/feathern/vis/demo2/ra_vis_demo_data > /dev/null
rm -rf /home/feathern/vis/demo2/ra_vis_demo.vdf > /dev/null
rm -rf /home/feathern/vis/demo2/ra_vis_demo_data > /dev/null
rm -rf /home/feathern/vis/demo2/ra_vis_demo_data > /dev/null


In [32]:
test.populate_dataset()

Converting data for timestep 0 of 1
Cleaning up temporary files
Complete.
